檢查有無安裝 boto3

```
!pip list | grep boto
```

沒有的話就 install 一下。

```
pip install boto3
```

## list objects in the bucket


In [6]:
import boto3
import re

In [6]:
# confige
s3_access_key = 'AKIAVF******P5W5XB'
s3_secret_key = '8K1oUcrOA*************CYD8kMEBpA4M'
name_bucket = 'sagemaker-us-west-2-355444812467'

In [7]:
s3 = boto3.client('s3',
                  aws_access_key_id=s3_access_key,
                  aws_secret_access_key=s3_secret_key)

In [8]:
bkts = s3.list_buckets()
bkts['Buckets']

[{'Name': 'crawl-voice',
  'CreationDate': datetime.datetime(2021, 7, 12, 15, 47, 45, tzinfo=tzutc())},
 {'Name': 'sagemaker-us-west-2-355444812467',
  'CreationDate': datetime.datetime(2021, 7, 2, 14, 46, 37, tzinfo=tzutc())}]

In [9]:
# list objects
objs = s3.list_objects_v2(Bucket = name_bucket, MaxKeys=3)
[x['Key'] for x in objs['Contents']]

['DEMO-tensorflow-cifar10/eval.tfrecords',
 'DEMO-tensorflow-cifar10/train.tfrecords',
 'DEMO-tensorflow-cifar10/validation.tfrecords']

In [12]:
# list all object
objs = []
tmp_objs = s3.list_objects_v2(Bucket = name_bucket, MaxKeys=1000)
objs = objs + [x['Key'] for x in tmp_objs['Contents']]
while 'NextContinuationToken' in tmp_objs.keys():
    tmp_objs = s3.list_objects_v2(Bucket = name_bucket, MaxKeys=1000, ContinuationToken = tmp_objs['NextContinuationToken'])
    objs = objs + [x['Key'] for x in tmp_objs['Contents']]
    
len(objs)

2836

In [17]:
# filter all wav files
p = re.compile('tomofun.+\.(wav|mp3)')
objs_filter = [ x for x in objs if p.match(x) ]

len(objs_filter)

13

In [18]:
objs_filter

['tomofun/1625340100.9683986/competition/train/train_00001.wav',
 'tomofun/1625340100.9683986/competition/train/train_00010.wav',
 'tomofun/1625340100.9683986/competition/train/train_00021.wav',
 'tomofun/1625346339.8631585/competition/train/train_00001.wav',
 'tomofun/1625346339.8631585/competition/train/train_00010.wav',
 'tomofun/1625346339.8631585/competition/train/train_00021.wav',
 'tomofun/1626372118.2400548/competition/train/train_00001.wav',
 'tomofun/1626372118.2400548/competition/train/train_00010.wav',
 'tomofun/1626372118.2400548/competition/train/train_00021.wav',
 'tomofun/1626434308.4505794/competition/train/28bdec5d-bac8-4463-9e5e-3a4c8241af9e.wav',
 'tomofun/1626434308.4505794/competition/train/train_00001.wav',
 'tomofun/1626434308.4505794/competition/train/train_00010.wav',
 'tomofun/1626434308.4505794/competition/train/train_00021.wav']

In [148]:
folder = {}
for i in objs_filter:
    tmp_re = re.search(pattern = 'category_[0-9]{2}', string = i)
    fname = tmp_re.group()
    tmp_url = 'https://crawl-voice.s3.us-east-1.amazonaws.com/'+i
    if fname not in folder.keys():
        folder[fname] = [tmp_url]
    else:
        folder[fname] = folder[fname] + [tmp_url]

In [149]:
folder.keys()

dict_keys(['category_00', 'category_01', 'category_02', 'category_04', 'category_05', 'category_06', 'category_07', 'category_09'])

In [ ]:
with open('filename.pickle', 'wb') as handle:
    pickle.dump(folder, handle, protocol=pickle.HIGHEST_PROTOCOL)

---

## END